In [1]:
import pandas as pd

# Charger les fichiers
sales = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Sales.xlsx")
products = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Products_f.xlsx")
sections = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Sections.xlsx")
customers = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Customers_f.xlsx")
colors = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Colors.xlsx")
product_qualities = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Product_qualities.xlsx")
reviews = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Reviews.xlsx")
shipping_types = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Shipping_types.xlsx")
pricing_strategies = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Pricing_strategies.xlsx")
payment_methods = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Payment_methods.xlsx")
orders = pd.read_excel(r"C:\Users\GSI\Desktop\DataWareHouse\Orders.xlsx")



print("\nColonnes de Customers_f.xlsx :")
print(customers.columns)

print("\nColonnes de Colors.xlsx :")
print(colors.columns)

print("\nColonnes de Product_qualities.xlsx :")
print(product_qualities.columns)

print("\nColonnes de Reviews.xlsx :")
print(reviews.columns)

print("\nColonnes de Shipping_types.xlsx :")
print(shipping_types.columns)

print("\nColonnes de Pricing_strategies.xlsx :")
print(pricing_strategies.columns)

print("\nColonnes de Payment_methods.xlsx :")
print(payment_methods.columns)

print("\nColonnes de Orders.xlsx :")
print(orders.columns)



Colonnes de Customers_f.xlsx :
Index(['code_customer', 'Age', 'Gender', 'Zip_code', 'Preferred_size',
       'Overall_review', 'Subscription Status', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

Colonnes de Colors.xlsx :
Index(['colour_group_code', 'colour_group_name', 'perceived_colour_value_id',
       'perceived_colour_value_name', 'perceived_colour_master_id',
       'perceived_colour_master_name', 'code_color'],
      dtype='object')

Colonnes de Product_qualities.xlsx :
Index(['code_quality', 'code_review_type', 'code_isrec_type', 'sku',
       'prod_name', 'quality'],
      dtype='object')

Colonnes de Reviews.xlsx :
Index(['code_review_type', 'review_text', 'stars'], dtype='object')

Colonnes de Shipping_types.xlsx :
Index(['shipping_type', 'ship_type_code'], dtype='object')

Colonnes de Pricing_strategies.xlsx :
Index(['code_pricing_strategy', 'category', 'brand', 'original_price',
       'markdown_percentage', 'current_pric

In [2]:
sales["sale_date"] = pd.to_datetime(sales["sale_date"])

cutoff_date = sales["sale_date"].quantile(0.7)    # 70% pour historique, 30% pour futur
print("Cutoff date =", cutoff_date)

historical = sales[sales["sale_date"] <= cutoff_date]
future = sales[sales["sale_date"] > cutoff_date]


Cutoff date = 2024-01-03 00:00:00


In [3]:
target_df = future.groupby("code_customer").agg(
    future_total_amount = ("Estimated_Unit_Price", "sum"),
    future_n_purchases = ("code_order", "nunique"),
    future_avg_basket = ("Estimated_Unit_Price", "mean")
).reset_index()

target_df.head()


,code_customer,future_total_amount,future_n_purchases,future_avg_basket
0,1,610.57,40,14.199302
1,2,702.51,46,14.336939
2,3,682.62,35,17.963684
3,4,660.56,43,15.012727
4,5,759.59,40,18.526585


In [4]:
customer_hist = historical.groupby("code_customer").agg(
    hist_total_amount = ("Estimated_Unit_Price", "sum"),
    hist_n_purchases = ("code_order", "nunique"),
    hist_avg_basket = ("Estimated_Unit_Price", "mean")
).reset_index()


In [5]:
features = customer_hist.merge(
    customers, on="code_customer", how="left"
)


In [6]:
sales_prod = historical.merge(products, on="SKU", how="left")



In [7]:
prod_stats = sales_prod.groupby("code_customer").agg(
    avg_original_price = ("Estimated_Unit_Price", "mean"),
    avg_quantity = ("quantity", "mean"),
).reset_index()

features = features.merge(prod_stats, on="code_customer", how="left")


In [8]:
df_ml = features.merge(target_df, on="code_customer", how="left")
print("Shape final :", df_ml.shape)
df_ml.head()


Shape final : (3900, 18)


,code_customer,hist_total_amount,hist_n_purchases,hist_avg_basket,Age,Gender,Zip_code,Preferred_size,Overall_review,Subscription Status,Previous Purchases,Payment Method,Frequency of Purchases,avg_original_price,avg_quantity,future_total_amount,future_n_purchases,future_avg_basket
0,1,1756.02,90,17.918571,55,Male,Lexington_Kentucky,L,3.1,Yes,14,Venmo,Fortnightly,17.918571,1.397959,610.57,40,14.199302
1,2,1360.07,86,14.624409,19,Male,Auburn_Maine,L,3.1,Yes,2,Cash,Fortnightly,14.624409,1.473118,702.51,46,14.336939
2,3,1107.31,65,16.047971,50,Male,Springfield_Massachusetts,S,3.1,Yes,23,Credit Card,Weekly,16.047971,1.333333,682.62,35,17.963684
3,4,1672.99,88,16.564257,21,Male,Cranston_Rhode Island,M,3.5,Yes,49,PayPal,Weekly,16.564257,1.425743,660.56,43,15.012727
4,5,1470.11,79,16.518090,45,Male,Eugene_Oregon,M,2.7,Yes,31,PayPal,Annually,16.518090,1.449438,759.59,40,18.526585


In [9]:
df_ml = df_ml.dropna(subset=["future_avg_basket"])  # garder clients avec target
df_ml = df_ml.fillna(0)  # simple, efficace


In [10]:
from sklearn.model_selection import train_test_split

X = df_ml.drop(["future_avg_basket"], axis=1).select_dtypes(include=['number'])
y = df_ml["future_avg_basket"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [11]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

print("Score train :", model.score(X_train, y_train))
print("Score test  :", model.score(X_test, y_test))


Score train : 0.9821191945520812
Score test  : 0.8809533391111597


In [12]:
preds = model.predict(X_test)
print(preds[:10])


[19.18391364 18.71669126 19.34189226 17.30286011 17.39599543 17.67697476
 14.22863976 17.65642463 19.61529281 15.2333863 ]


In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Prédictions sur le test set
y_pred = model.predict(X_test)

# Calcul des métriques
mae  = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2   = r2_score(y_test, y_pred)

print("📌 Résultats du modèle")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R²   : {r2:.4f}")


📌 Résultats du modèle
MAE  : 0.4549
RMSE : 0.5863
R²   : 0.8810


c:\Users\GSI\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
# Sauvegarder les informations pour l'intégration Django
import joblib
import os

# Créer le dossier models s'il n'existe pas
models_dir = '../models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Identifier les features utilisées pour l'entraînement
feature_cols = X_train.columns.tolist()
print("Features d'entraînement:", feature_cols)

# Features de base (directes depuis le formulaire)
base_features = [
    'Age', 'Overall_review', 'Previous Purchases', 'Subscription Status',
    'Payment Method', 'Frequency of Purchases', 'Preferred_size'
]

# Features dérivées (calculées depuis l'historique des ventes)
derived_features = [
    'hist_total_amount',    # Somme des prix des achats historiques
    'hist_n_purchases',     # Nombre de commandes historiques
    'hist_avg_basket',      # Moyenne des prix des achats historiques
    'avg_original_price',   # Moyenne des prix unitaires historiques
    'avg_quantity'          # Moyenne des quantités historiques
]

print("\nFeatures de base (directes):", base_features)
print("Features dérivées (calculées):", derived_features)

# Sauvegarder les informations des features pour Django
features_info = {
    'feature_cols': feature_cols,
    'base_features': base_features,
    'derived_features': derived_features,
    'target_column': 'future_avg_basket'
}

joblib.dump(features_info, os.path.join(models_dir, 'future_avg_basket_features.pkl'))
print(f"\n✅ Informations des features sauvegardées dans {models_dir}/future_avg_basket_features.pkl")

# Pour les encodeurs - ce modèle n'utilise que des features numériques
# donc pas d'encodeurs catégoriels nécessaires
print("ℹ️ Ce modèle n'utilise que des features numériques - pas d'encodeurs catégoriels requis")

print("\n📋 Guide pour l'intégration Django:")
print("1. Les features de base viennent directement du formulaire")
print("2. Les features dérivées doivent être calculées depuis l'historique des ventes du client")
print("3. Utiliser les mêmes formules d'agrégation que dans ce notebook")
print("4. Charger le modèle depuis 'future_avg_basket_model.pkl'")
print("5. Charger les features_info depuis 'future_avg_basket_features.pkl'")

Features d'entraînement: ['code_customer', 'hist_total_amount', 'hist_n_purchases', 'hist_avg_basket', 'Age', 'Overall_review', 'Previous Purchases', 'avg_original_price', 'avg_quantity', 'future_total_amount', 'future_n_purchases']

Features de base (directes): ['Age', 'Overall_review', 'Previous Purchases', 'Subscription Status', 'Payment Method', 'Frequency of Purchases', 'Preferred_size']
Features dérivées (calculées): ['hist_total_amount', 'hist_n_purchases', 'hist_avg_basket', 'avg_original_price', 'avg_quantity']

✅ Informations des features sauvegardées dans ../models/future_avg_basket_features.pkl
ℹ️ Ce modèle n'utilise que des features numériques - pas d'encodeurs catégoriels requis

📋 Guide pour l'intégration Django:
1. Les features de base viennent directement du formulaire
2. Les features dérivées doivent être calculées depuis l'historique des ventes du client
3. Utiliser les mêmes formules d'agrégation que dans ce notebook
4. Charger le modèle depuis 'future_avg_basket_mo